In [ ]:
import pandas as pd
import glob
from igc_lib.igc_lib import Flight
import pdflight
from joblib import Parallel, delayed
import matplotlib
import numpy as np

%matplotlib inline

In [ ]:
flights = Parallel(n_jobs=-1)(
    delayed(Flight.create_from_file)(i) for i in glob.glob("PC2011/*.igc")
)
# flight = Flight.create_from_file("/home/muenchm/thinclient_drives/C:/Users/muenchm/Desktop/2025-05-01 Wolfgang Janowitsch 562460.igc")

In [ ]:
md = pd.concat(pdflight.flight_to_metadata(flight) for flight in flights if flight.valid and flight.task_start_time).sort_index()
md

In [ ]:
fl = pd.concat(
    Parallel(n_jobs=-1)(
        delayed(pdflight.flight_to_dataframe)(flight) for flight in flights if flight.valid and flight.task_start_time
    )
).sort_index()
fl

In [ ]:
th = (
    pd.concat(pdflight.thermals_to_dataframe(flight) for flight in flights if flight.valid and flight.task_start_time)
    .sort_index()
    .loc[fl.loc[fl.index, "task"]]
)
th.dt.total_seconds().plot(kind="hist", bins=16)
th = th[th.dt.total_seconds() >= 240]
th

In [ ]:
def shift_datetime(i, dt):
    return ((i[0] + dt),) + i[1:]

In [ ]:
df = pd.DataFrame()
df["t"] = th.index.get_level_values("datetime") + th
df = df.sort_index()
for i in th.index:
    df.loc[i, "v"] = (
        fl.loc[shift_datetime(i, th.loc[i] + pd.Timedelta(seconds=40))].alt
        - fl.loc[i].alt
    ) / (th.loc[i].total_seconds() - 40)
    df.loc[i, "n"] = (
        fl.loc[shift_datetime(i, pd.Timedelta(seconds=40))].alt - fl.loc[i].alt
    ) / 40
    df.loc[i, "x"] = (
        fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=60))].alt
        - fl.loc[shift_datetime(i, th.loc[i] - pd.Timedelta(seconds=20))].alt
    ) / -40
df["dn"] = df.n / df.v
df["dx"] = df.x / df.v

In [ ]:
dn = df[(df.v >= 1) & (df.dn < 1.5)].groupby(["code", "id", "date"]).dn.mean()
dn

In [ ]:
cc = pd.concat([dn, md.points], axis=1).dropna().sort_values(by="dn")
cc.plot(x="dn", y="points", kind="scatter")
cc

In [ ]:
from scipy import stats

# Berechnet Steigung, Schnittpunkt UND den Korrelationskoeffizienten (r)
stats.linregress(np.log(cc.dn), cc.points)
